In [1]:
import os
import json
import ray
import numpy as np
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
from flow.networks import Network
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams, VehicleParams, SumoCarFollowingParams, InFlows, SumoLaneChangeParams
from flow.controllers import RLController, IDMController, SimLaneChangeController
from gym.spaces.box import Box
from gym.spaces import Tuple

from constructionRouter import ConstructionRouter
from constructionEnv_simplifiedV2 import myEnv

# time horizon of a single rollout
HORIZON = 5000 #5000
# number of rollouts per training iteration
N_ROLLOUTS = 10
# number of parallel workers
N_CPUS = 1

vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             routing_controller=(ConstructionRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
             ),
             num_vehicles=8
             )
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode=25
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode=1621),
             num_vehicles=0)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["edge4"]
)

# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "edge1": ["edge1","edge2","edge3","edge4","edge5","edge6"],
                #"edge3": ["edge3","edge4","edge5","edge10","edge11","edge12","edge3"],
                "edge4": ["edge4","edge5","edge10","edge11","edge12","edge3","edge4"],
                }


inflow = InFlows()
inflow.add(veh_type="human",
           edge="edge1",
           vehs_per_hour=3000,
            depart_lane="random",
            depart_speed="random",
            color="white")

file_dir = "/home/llavezzo/"
net_params = NetParams(
    template="/mnt/c/users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/construction/constructionV6.net.xml",
    inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="construction_traffic_simplified",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.5,
        render=False,
        restart_instance=True,
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=1000,
        clip_actions=False,
        additional_params={
            "target_velocity": 50,
            "sort_vehicles": False,
            "max_accel": 2,
            "max_decel": 2,
            "lane_change_duration": 5,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [32, 32, 32, 32]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config




/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [ ]:
alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 5,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})

2020-04-26 17:47:18,994	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-04-26 17:47:18,999	INFO resource_spec.py:212 -- Starting Ray with 4.69 GiB memory available for workers and up to 2.36 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-04-26 17:47:19,613	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_819057b6,RUNNING,


(pid=raylet) E0426 17:47:19.977710000     668 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1587937639.977672500","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=raylet) E0426 17:47:19.978279700     668 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet) E0426 17:47:20.016371500     668 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=690) E0426 17:47:22.153048700     690 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1587937642.153013600","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=69

(pid=689) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=689)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=689) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=689)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=689) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=689)   _np_qint16 = np.dtype([("qint16", np.int16, 1)])
(pid

(pid=689) Warning: Vehicle 'flow_00.238' performs emergency braking with decel=-27.36 wished=4.50 severity=5.08, time=569.50.
(pid=689) Warning: Vehicle 'flow_00.233' performs emergency braking with decel=-17.08 wished=4.50 severity=2.80, time=570.00.
(pid=689) Warning: Vehicle 'flow_00.218' performs emergency braking with decel=-26.44 wished=4.50 severity=4.87, time=607.50.
(pid=689) Warning: Vehicle 'flow_00.225' performs emergency braking with decel=-25.81 wished=4.50 severity=4.73, time=608.00.
(pid=689) Warning: Vehicle 'flow_00.237' performs emergency braking with decel=-13.12 wished=4.50 severity=1.92, time=608.50.
(pid=689) Warning: Vehicle 'flow_00.252' performs emergency braking with decel=-25.62 wished=4.50 severity=4.69, time=641.50.
(pid=689) Warning: Vehicle 'flow_00.251' performs emergency braking with decel=-14.25 wished=4.50 severity=2.17, time=642.00.
(pid=689) Warning: Teleporting vehicle 'flow_00.273'; collision with vehicle 'rl_2', lane=':gneJ31_1_0', gap=-1.00, ti

...

(pid=689) Warning: Vehicle 'rl_0' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=498.00.
(pid=689) Warning: Vehicle 'flow_00.186' performs emergency braking with decel=-14.56 wished=4.50 severity=2.24, time=499.50.
(pid=689) Warning: Vehicle 'flow_00.189' performs emergency braking with decel=-13.99 wished=4.50 severity=2.11, time=500.00.
(pid=689) Warning: Vehicle 'flow_00.187' performs emergency braking with decel=-13.55 wished=4.50 severity=2.01, time=500.50.
(pid=689) /mnt/c/Users/llave/Documents/GitHub/flow/flow/core/kernel/vehicle/traci.py:957: UserWarning: API change now handles duration as floating point seconds
(pid=689)   veh_id, int(target_lane), 100000)
(pid=689) Warning: Vehicle 'flow_00.190' performs emergency braking with decel=-13.27 wished=4.50 severity=1.95, time=501.00.
(pid=689) Warning: Vehicle 'flow_00.191' performs emergency braking with decel=-13.10 wished=4.50 severity=1.91, time=501.50.
(pid=689) Warning: Vehicle 'flow_00.197' perf

(pid=689) Warning: Vehicle 'flow_00.28' performs emergency braking with decel=-10.70 wished=4.50 severity=1.38, time=157.00.
(pid=689) Warning: Vehicle 'flow_00.108' performs emergency braking with decel=-13.35 wished=4.50 severity=1.97, time=267.50.
(pid=689) Warning: Vehicle 'flow_00.116' performs emergency braking with decel=-20.77 wished=4.50 severity=3.61, time=280.00.
(pid=689) Warning: Vehicle 'flow_00.144' performs emergency braking with decel=-17.94 wished=4.50 severity=2.99, time=308.00.
(pid=689) Warning: Vehicle 'flow_00.87' performs emergency braking with decel=-9.14 wished=4.50 severity=1.03, time=315.50.
(pid=689) Warning: Vehicle 'flow_00.253' performs emergency braking with decel=-11.65 wished=4.50 severity=1.59, time=476.00.
(pid=689) Warning: Vehicle 'flow_00.254' performs emergency braking with decel=-9.13 wished=4.50 severity=1.03, time=476.50.
(pid=689) Warning: Vehicle 'flow_00.229' performs emergency braking with decel=-9.81 wished=4.50 severity=1.18, time=479.5

(pid=689) Warning: Vehicle 'flow_00.206' performs emergency braking with decel=-9.74 wished=4.50 severity=1.16, time=508.00.
(pid=689) Warning: Vehicle 'rl_7' performs emergency braking with decel=-14.86 wished=4.50 severity=2.30, time=658.00.
(pid=689) Warning: Vehicle 'flow_00.224' performs emergency braking with decel=-17.37 wished=4.50 severity=2.86, time=658.50.
(pid=689) Warning: Vehicle 'flow_00.160' performs emergency braking with decel=-23.72 wished=4.50 severity=4.27, time=962.50.
(pid=689) Warning: Vehicle 'flow_00.290' performs emergency braking with decel=-22.34 wished=4.50 severity=3.96, time=963.00.
(pid=689) Warning: Vehicle 'flow_00.293' performs emergency braking with decel=-23.30 wished=4.50 severity=4.18, time=963.50.
(pid=689) Warning: Teleporting vehicle 'flow_00.289'; collision with vehicle 'rl_6', lane=':gneJ30_2_1', gap=-1.00, time=1017.00 stage=move.
(pid=689) Warning: Vehicle 'rl_7' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=9

(pid=689) Warning: Vehicle 'flow_00.219' performs emergency braking with decel=-14.87 wished=4.50 severity=2.31, time=512.50.
(pid=689) Warning: Vehicle 'flow_00.400' performs emergency braking with decel=-10.59 wished=4.50 severity=1.35, time=776.00.
(pid=689) Warning: Vehicle 'flow_00.403' performs emergency braking with decel=-13.32 wished=4.50 severity=1.96, time=776.50.
(pid=689) Warning: Vehicle 'flow_00.405' performs emergency braking with decel=-17.27 wished=4.50 severity=2.84, time=777.00.
(pid=689) Warning: Vehicle 'flow_00.407' performs emergency braking with decel=-18.16 wished=4.50 severity=3.04, time=777.50.
(pid=689) Warning: Vehicle 'flow_00.410' performs emergency braking with decel=-18.25 wished=4.50 severity=3.05, time=778.00.
(pid=689) Warning: Vehicle 'flow_00.406' performs emergency braking with decel=-18.25 wished=4.50 severity=3.06, time=778.50.
(pid=689) Warning: Vehicle 'flow_00.411' performs emergency braking with decel=-18.23 wished=4.50 severity=3.05, time=

(pid=689) Warning: Vehicle 'flow_00.199' performs emergency braking with decel=-9.51 wished=4.50 severity=1.11, time=406.50.
(pid=689) Warning: Vehicle 'flow_00.179' performs emergency braking with decel=-9.27 wished=4.50 severity=1.06, time=407.00.
(pid=689) Warning: Vehicle 'flow_00.196' performs emergency braking with decel=-19.09 wished=4.50 severity=3.24, time=452.50.
(pid=689) /mnt/c/Users/llave/Documents/GitHub/flow/flow/core/kernel/vehicle/traci.py:957: UserWarning: API change now handles duration as floating point seconds
(pid=689)   veh_id, int(target_lane), 100000)
(pid=689) Warning: Vehicle 'rl_7' performs emergency braking with decel=-16.52 wished=4.50 severity=2.67, time=527.00.
(pid=689) Warning: Vehicle 'flow_00.218' performs emergency braking with decel=-11.21 wished=4.50 severity=1.49, time=527.50.
(pid=689) Warning: Vehicle 'flow_00.215' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=529.00.
(pid=689) Warning: Teleporting vehicle 'flow_00